In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

: 

In [15]:
# Define dataset path
dataset_path = "train"

In [16]:
from PIL import Image
import os

dataset_path = "train"  # Change this to your dataset directory

def check_and_remove_corrupt_images(directory):
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            for img_file in os.listdir(category_path):
                img_path = os.path.join(category_path, img_file)
                try:
                    with Image.open(img_path) as img:
                        img.verify()  # Check if the image is valid
                except (OSError, IOError):
                    print(f"Corrupt image removed: {img_path}")
                    os.remove(img_path)  # Delete the corrupt image

check_and_remove_corrupt_images(dataset_path)

In [17]:
# Image parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [26]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [27]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Fix truncated image loading

# Now define ImageDataGenerator and continue your training
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)


In [28]:
# Data augmentation and loading
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)
val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 11576 images belonging to 8 classes.
Found 2891 images belonging to 8 classes.


In [29]:
# Load pre-trained MobileNetV2 model as feature extractor
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

In [30]:
# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output_layer = Dense(len(train_data.class_indices), activation='softmax')(x)

In [31]:
model = Model(inputs=base_model.input, outputs=output_layer)

In [32]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [34]:
# Train the model
epochs = 3
history = model.fit(train_data, validation_data=val_data, epochs=epochs)

Epoch 1/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 308s 842ms/step - accuracy: 0.7280 - loss: 0.9038 - val_accuracy: 0.8547 - val_loss: 0.4483
Epoch 2/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 394s 1s/step - accuracy: 0.9366 - loss: 0.2070 - val_accuracy: 0.8796 - val_loss: 0.4099
Epoch 3/3
362/362 ━━━━━━━━━━━━━━━━━━━━ 599s 2s/step - accuracy: 0.9537 - loss: 0.1431 - val_accuracy: 0.8789 - val_loss: 0.3959


In [35]:
model.save("waste_classifier.h5")

In [1]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("waste_classifier.h5")

# Convert to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open("waste_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved!")

INFO:tensorflow:Assets written to: C:\Users\Dhruv\AppData\Local\Temp\tmp35v30s2z\assets


INFO:tensorflow:Assets written to: C:\Users\Dhruv\AppData\Local\Temp\tmp35v30s2z\assets


Saved artifact at 'C:\Users\Dhruv\AppData\Local\Temp\tmp35v30s2z'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  2342429886992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429887760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429888720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429887184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429888336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429887952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429887376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429888144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429889296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342429890064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2342